In [1]:
import os
import json
import joblib
import numpy as np
import pandas as pd
from pathlib import Path
import flwr as fl

# ===== 参数设置 =====
SERVER       = "127.0.0.1:8080"
FEATURES_CSV = "wsi_stage_features_topk_4class_split.csv"   # 包含所有病人特征的文件
MODEL_PATH   = "new_stage_classifier_gbdt.pkl"           # 你训练好的模型
MODALITY     = "WSI"
N_CLASSES    = 4
WEIGHT       = 1.0

# ===== 自动获取 test 病人列表（从 .svs 文件名中提取）=====
TEST_IMG_DIR = r"C:\Users\zxy01\Desktop\svs"
test_pids = [
    "-".join(os.path.splitext(f)[0].split("-")[:3])
    for f in os.listdir(TEST_IMG_DIR)
    if f.endswith(".svs")
]
print(f"[INFO] Found {len(test_pids)} test patients.")

# ===== 读取特征 CSV 和模型 =====
df = pd.read_csv(FEATURES_CSV)
model = joblib.load(MODEL_PATH)

# ===== 提取特征列（所有 cnn_ 开头 + 其他人工特征）=====
feature_cols = [c for c in df.columns if c.startswith("cnn_")] + [
    "tumor_frac", "largest_cc_frac", "cc_count", "cc_small_frac", "frag_ratio"
]

# ===== 提取测试数据（根据 patient_id 匹配）=====
df["pid"] = df["path"].apply(lambda x: "-".join(Path(x).stem.split("-")[:3]))
df_test = df[df["pid"].isin(test_pids)].copy()
df_test = df_test.drop_duplicates("pid") 
x_test = df_test[feature_cols].values
pid_list = df_test["pid"].tolist()
print(f"[INFO] Matched {len(pid_list)} patients with features.")

# ===== 构造联邦客户端：输出 softmax =====
class SoftmaxClient(fl.client.NumPyClient):
    def __init__(self, pids, features, model, modality, weight, n_classes):
        self.pids = pids
        self.x = features
        self.model = model
        self.modality = modality
        self.weight_for_fusion = float(weight)
        self.n_classes = n_classes

    def get_parameters(self, config):
        return []

    def fit(self, parameters, config):
        return [], 0, {}

    def evaluate(self, parameters, config):
        task = config.get("task", "")
        metrics = {}

        if task == "predict":
            rows = []
            probs_all = self.model.predict_proba(self.x)
            print(f"[INFO] Predicting for {len(self.pids)} patients...")

            for pid, probs in zip(self.pids, probs_all):
                probs = np.clip(probs.astype(float), 1e-9, 1.0)
                probs = probs / probs.sum()
                row = {
                    "patient_id": pid,
                    "probs": probs.tolist(),
                    "modality": self.modality,
                    "weight": self.weight_for_fusion
                }
                print(f"[PREDICT] {row}")  # ✅ 正确：现在 row 有定义
                rows.append(row)

            metrics = {"preds_json": json.dumps(rows).encode("utf-8")}
            print(f"[INFO] Finished prediction. Sent {len(rows)} results to server.")

        return 0.0, len(self.pids), metrics

[INFO] Found 20 test patients.
[INFO] Matched 20 patients with features.


In [2]:
# ===== 启动联邦客户端 =====
client = SoftmaxClient(pid_list, x_test, model, MODALITY, WEIGHT, N_CLASSES)
fl.client.start_numpy_client(server_address=SERVER, client=client)

	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message 126c9225-371c-40f2-8169-57eccb342e75
INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 3e8e2024-6

[INFO] Predicting for 20 patients...
[PREDICT] {'patient_id': 'TCGA-A2-A04N', 'probs': [0.997132319267581, 0.0003141789444394046, 0.002405289189611794, 0.00014821259836774866], 'modality': 'WSI', 'weight': 1.0}
[PREDICT] {'patient_id': 'TCGA-A2-A04R', 'probs': [0.9960244582618463, 0.002914198636091929, 0.0005468330096172574, 0.0005145100924446099], 'modality': 'WSI', 'weight': 1.0}
[PREDICT] {'patient_id': 'TCGA-A2-A0T3', 'probs': [0.9967437845333696, 0.0015624140805702593, 0.0015078585145455333, 0.00018594287151463138], 'modality': 'WSI', 'weight': 1.0}
[PREDICT] {'patient_id': 'TCGA-A2-A3XZ', 'probs': [0.996105920569524, 0.00281130159105442, 0.0008478151243569298, 0.00023496271506457986], 'modality': 'WSI', 'weight': 1.0}
[PREDICT] {'patient_id': 'TCGA-A7-A26E', 'probs': [0.9972449587699398, 0.001405317273501436, 0.0009502539396642957, 0.00039947001689438863], 'modality': 'WSI', 'weight': 1.0}
[PREDICT] {'patient_id': 'TCGA-A7-A426', 'probs': [0.5270273223601764, 0.001833717093735818